In [1]:
import requests
import json
import io
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from IPython.display import clear_output
from bokeh.io import show, output_file, save, output_notebook, export_png
from bokeh.plotting import figure
from bokeh.palettes import Cividis256
from bokeh.transform import factor_cmap
from bokeh.models import ColumnDataSource
from bokeh.resources import CDN
from bokeh.embed import file_html

In [2]:
output_notebook()

Loading BokehJS ...

### Get neighborhood boundaries 

In [3]:
# NYC neighborhood geojson from insideairbnb.com
nycNeigh = gpd.read_file('http://data.insideairbnb.com/united-states/ny/new-york-city/2019-06-02/visualisations/neighbourhoods.geojson')
nycNeigh.shape

(233, 3)

In [4]:
nycNeigh['neighCenter'] = nycNeigh.geometry.centroid
nycNeigh['search_query'] = nycNeigh['neighbourhood'] + ', ' + nycNeigh['neighbourhood_group'] + ', NY'
nycNeigh.head()

,neighbourhood,neighbourhood_group,geometry,neighCenter,search_query
0,Bayswater,Queens,"(POLYGON ((-73.76670799999999 40.614911, -73.7...",POINT (-73.769503479281 40.61740017645801),"Bayswater, Queens, NY"
1,Allerton,Bronx,"(POLYGON ((-73.848597 40.87167, -73.845823 40....",POINT (-73.8599842270222 40.86472735568424),"Allerton, Bronx, NY"
2,City Island,Bronx,"(POLYGON ((-73.782822 40.843919, -73.782572 40...",POINT (-73.78665350533366 40.84693161668218),"City Island, Bronx, NY"
3,Ditmars Steinway,Queens,"(POLYGON ((-73.90160299999999 40.76777, -73.90...",POINT (-73.90600334689506 40.77780367906778),"Ditmars Steinway, Queens, NY"
4,Ozone Park,Queens,"(POLYGON ((-73.83754399999999 40.691364, -73.8...",POINT (-73.84636439104509 40.67908264800706),"Ozone Park, Queens, NY"


### Show on map

In [5]:
# basemap
NYC_COORDINATES = (40.7128, -74.0060)
nycMap = folium.Map(location=NYC_COORDINATES, tiles='cartodbpositron', zoom_start=10)


# add neighborhood boundaries
folium.GeoJson(
    nycNeigh['geometry'],
    name='geojson',
    style_function=lambda feature: {
        'fillColor': None,
        'color' : 'gray',
        'weight' : 1,
        'fillOpacity' : 0.3,
        }).add_to(nycMap)


# add points
for i, loc in enumerate(nycNeigh.neighCenter):
    folium.Circle(radius=200, 
                  location=[loc.y, loc.x], 
                  popup=nycNeigh['neighbourhood'][i],
                  color='crimson',
                  fill=True).add_to(nycMap)
    
    
nycMap

Map showing the boundaries of each neighborhood and the neighborhood centroid.

In [6]:
nycMap.save('nyc_neighborhoods.html')

### Crawl Yelp API

In [7]:
def getYelpData(apiKey, businessCat, location):
    
    """Calls Yelp API and returns a list with business information in the given location.
    
    Args:
        api_key (str): Yelp API key.
        business_cat (str): Business category for request.
        location (str): Location of interest (e.g. East Village, NY).
    
    Returns:
        dataframe : Dataframe including all available business information (e.g. name, rating, address, etc.)
        
    """
    
    # authorize API call
    headers = {'Authorization': 'Bearer %s' % apiKey}
    
    # API url
    url='https://api.yelp.com/v3/businesses/search'
    
    # set request parameters
    params = {'term': businessCat, 'location': location, 'limit': 50}
    
    # make the request
    req = requests.get(url, params=params, headers=headers)
    
    # read request as json
    j = req.json()
    
    # convert to dataframe
    df = pd.DataFrame(j['businesses'])
    
    return df

In [8]:
# Yelp API key. Get yours here: https://www.yelp.com/developers/v3/manage_app
f = open("yelp-api-key.json", "r")
apiKey = f.read()

#### Request restaurant info for all neighborhoods

In [9]:
data = pd.DataFrame()

for idx, neighbourhood in enumerate(nycNeigh['search_query']):
    
    # call API for a neighborhood
    yelp_data = getYelpData(apiKey, 'restaurants', neighbourhood)
    yelp_data['neighbourhood'] = neighbourhood
    
    # put all data together 
    data = pd.concat([data, yelp_data]) 
    
    if idx in np.arange(0, len(nycNeigh['search_query']), 20):
        clear_output()
        print ("Processed %s / %s examples" %(idx,str(len(nycNeigh['search_query']))))

Processed 220 / 233 examples


In [10]:
data.shape

(11535, 17)

In [11]:
data.head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url,neighbourhood
0,dredsurfer-grill-far-rockaway-2,"[{'alias': 'tradamerican', 'title': 'American ...","{'latitude': 40.594113, 'longitude': -73.750546}",(718) 471-1691,1853.713950,9EiSjoAmpySrzJjEQzwP7w,https://s3-media2.fl.yelpcdn.com/bphoto/rFCNBo...,False,"{'address1': '16-27 Seagirt Blvd', 'address2':...",DredSurfer Grill,+17184711691,$$,4.0,10,"[pickup, delivery]",https://www.yelp.com/biz/dredsurfer-grill-far-...,"Bayswater, Queens, NY"
1,tacos-y-mas-mexican-restaurant-far-rockaway,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...","{'latitude': 40.6029614, 'longitude': -73.7507...",(718) 327-3001,1384.496300,gMMwaw61b722ngFem5abhA,https://s3-media4.fl.yelpcdn.com/bphoto/CDkcSG...,False,"{'address1': '1831 Mott Ave', 'address2': '', ...",Tacos Y Mas Mexican Restaurant,+17183273001,$,3.5,53,"[pickup, delivery]",https://www.yelp.com/biz/tacos-y-mas-mexican-r...,"Bayswater, Queens, NY"
2,mamas-cooking-far-rockaway,"[{'alias': 'caribbean', 'title': 'Caribbean'},...","{'latitude': 40.603509, 'longitude': -73.753601}",(718) 327-3720,1135.135763,B2d_xUEt9QoC2DluIbeb3g,https://s3-media2.fl.yelpcdn.com/bphoto/BeaOfz...,False,"{'address1': '1044 Beach 21st St', 'address2':...",Mama's Cooking,+17183273720,NaN,4.5,3,[],https://www.yelp.com/biz/mamas-cooking-far-roc...,"Bayswater, Queens, NY"
3,dred-surfer-bar-far-rockaway,"[{'alias': 'caribbean', 'title': 'Caribbean'},...","{'latitude': 40.594113, 'longitude': -73.750547}",(646) 463-3271,1853.651289,eXSiOni-X1oUsxFPWypfrA,https://s3-media3.fl.yelpcdn.com/bphoto/JzhWSL...,False,"{'address1': '16-27 Seagirt Blvd', 'address2':...",Dred Surfer Bar,+16464633271,NaN,5.0,1,[],https://www.yelp.com/biz/dred-surfer-bar-far-r...,"Bayswater, Queens, NY"
4,marios-far-rockaway-2,"[{'alias': 'chicken_wings', 'title': 'Chicken ...","{'latitude': 40.6028743817863, 'longitude': -7...",(718) 471-5103,1406.522481,BQxIP160lPc1ido3TKbA9g,https://s3-media1.fl.yelpcdn.com/bphoto/ph4fgx...,False,"{'address1': '1813 Mott Ave', 'address2': '', ...",Mario's,+17184715103,$,4.5,10,"[pickup, delivery]",https://www.yelp.com/biz/marios-far-rockaway-2...,"Bayswater, Queens, NY"


In [12]:
data.to_csv(r'yelp_ratings.csv', index=False)

In [13]:
data.groupby('neighbourhood')['rating'].mean().sort_values(ascending=False)

neighbourhood
Sunnyside, Queens, NY                        4.370000
Bedford-Stuyvesant, Brooklyn, NY             4.330000
Greenpoint, Brooklyn, NY                     4.330000
Ridgewood, Queens, NY                        4.330000
Lower East Side, Manhattan, NY               4.320000
Ditmars Steinway, Queens, NY                 4.320000
Williamsburg, Brooklyn, NY                   4.310000
Astoria, Queens, NY                          4.300000
West Village, Manhattan, NY                  4.300000
Whitestone, Queens, NY                       4.290000
Two Bridges, Manhattan, NY                   4.290000
Flushing, Queens, NY                         4.280000
Prospect-Lefferts Gardens, Brooklyn, NY      4.280000
East Elmhurst, Queens, NY                    4.280000
Bushwick, Brooklyn, NY                       4.270000
Navy Yard, Brooklyn, NY                      4.270000
East Harlem, Manhattan, NY                   4.270000
Glendale, Queens, NY                         4.260000
Brownsville, B

In [14]:
avg_ratings = pd.DataFrame({'neighborhood': data.groupby('neighbourhood')['rating'].mean().sort_values(ascending=False).index,
                           'avg_rating':data.groupby('neighbourhood')['rating'].mean().sort_values(ascending=False).values})

### All together
Merge yelp data with neighborhood geojson file

In [15]:
yelp_ratings = avg_ratings.merge(nycNeigh, left_on='neighborhood', right_on='search_query')

In [16]:
yelp_ratings.head()

,neighborhood,avg_rating,neighbourhood,neighbourhood_group,geometry,neighCenter,search_query
0,"Sunnyside, Queens, NY",4.37,Sunnyside,Queens,"(POLYGON ((-73.91189 40.748715, -73.911642 40....",POINT (-73.92012593552306 40.74020523800707),"Sunnyside, Queens, NY"
1,"Bedford-Stuyvesant, Brooklyn, NY",4.33,Bedford-Stuyvesant,Brooklyn,"(POLYGON ((-73.941149 40.700281, -73.905479 40...",POINT (-73.9382015250131 40.68706821542903),"Bedford-Stuyvesant, Brooklyn, NY"
2,"Greenpoint, Brooklyn, NY",4.33,Greenpoint,Brooklyn,"(POLYGON ((-73.96226299999999 40.732916, -73.9...",POINT (-73.94607148318792 40.72762983871867),"Greenpoint, Brooklyn, NY"
3,"Ridgewood, Queens, NY",4.33,Ridgewood,Queens,"(POLYGON ((-73.90059599999999 40.712697, -73.9...",POINT (-73.90580187199851 40.70537908128151),"Ridgewood, Queens, NY"
4,"Lower East Side, Manhattan, NY",4.32,Lower East Side,Manhattan,"(POLYGON ((-73.97489299999999 40.715152, -73.9...",POINT (-73.98461604379614 40.71608729315376),"Lower East Side, Manhattan, NY"


### Choropleth

In [17]:
url = 'http://data.insideairbnb.com/united-states/ny/new-york-city/2019-06-02/visualisations'
neigh_geo = f'{url}/neighbourhoods.geojson'


# Initialize the map
m = folium.Map(location=NYC_COORDINATES, tiles='cartodbpositron', zoom_start=10)
 
    
# Define choropleth
choropleth = folium.Choropleth(geo_data=neigh_geo,
    data=yelp_ratings,
    bins=np.linspace(yelp_ratings.avg_rating.min(), yelp_ratings.avg_rating.max(), 10),
    name='yelp_ratings_choropleth',
    columns=['neighbourhood', 'avg_rating'],
    key_on='feature.properties.neighbourhood',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2).add_to(m)



# Find the neighborhood with highest average rating
maxRating = gpd.GeoDataFrame(yelp_ratings[yelp_ratings.avg_rating==yelp_ratings.avg_rating.max()])
maxRating.crs = {'init' :'epsg:4326'}
del maxRating['neighCenter']

folium.GeoJson(
    data=maxRating,
    name='highest_rated_neighborhood',
    style_function=lambda feature: {
        'fillColor': 'firebrick',
        'color' : 'gray',
        'weight' : 1,
        'fillOpacity' : 0.8,
        }
    ).add_to(m)


# hacky way to show popups in choropleth
for i, loc in enumerate(nycNeigh.neighCenter):
    folium.Circle(radius=1000, 
                  location=[loc.y, loc.x], 
                  popup=nycNeigh['neighbourhood'][i] + '\n AvgRating: ' +  
                         str(yelp_ratings[yelp_ratings.neighbourhood==nycNeigh['neighbourhood'][i]].avg_rating.values[0]),
                  color=None,
                  fill_opacity=0,
                  line_opacity=0,
                  fill=True).add_to(m)


# To control the layers shown
#folium.LayerControl().add_to(m)

m

In [18]:
m.save('yelp_choropleth.html')

### Cuisine with highest rating

In [19]:
data2 = data.copy()

In [20]:
print(data2.shape)
data2 = data2.drop_duplicates('id')
print(data2.shape)

(11535, 17)
(4694, 17)


In [21]:
# extract cuisine label
data2['cuisine'] = data2.categories.map(lambda x: x[0]['alias'])

In [22]:
# find top 30 cuisines in data
print(data2.groupby('cuisine')['rating'].count().sort_values(ascending=False).head(30).index.values)

top30 = data2[data2.cuisine.isin(data2.groupby('cuisine')['rating'].count().sort_values(ascending=False).head(30).index)]

['italian' 'pizza' 'mexican' 'chinese' 'newamerican' 'caribbean'
 'japanese' 'seafood' 'tradamerican' 'bars' 'delis' 'latin'
 'mediterranean' 'cafes' 'breakfast_brunch' 'burgers' 'thai' 'coffee'
 'sushi' 'indpak' 'korean' 'spanish' 'french' 'diners' 'sandwiches'
 'vietnamese' 'greek' 'bbq' 'ramen' 'wine_bars']


In [23]:
top30.groupby('cuisine')['rating'].mean().sort_values(ascending=False)

cuisine
wine_bars           4.256098
french              4.200000
cafes               4.195652
korean              4.176056
coffee              4.153846
mediterranean       4.132653
greek               4.115385
ramen               4.102041
newamerican         4.046729
italian             4.040741
indpak              4.034722
japanese            4.034615
thai                4.018987
mexican             4.010460
breakfast_brunch    3.994253
sushi               3.966667
vietnamese          3.947368
delis               3.945455
sandwiches          3.940678
bars                3.936441
bbq                 3.840000
caribbean           3.836879
pizza               3.835317
latin               3.805000
seafood             3.788462
chinese             3.779545
diners              3.685484
spanish             3.647059
burgers             3.640244
tradamerican        3.624031
Name: rating, dtype: float64

#### Interactive bar chart

In [24]:
# set custom colormap with 30 values from cividis256
cividis30=[]

for i in range(0, len(Cividis256), 8):
    cividis30.append(Cividis256[i])

cividis30 = cividis30[:30]


# variables to plot
cuisines = top30.groupby('cuisine')['rating'].mean().sort_values(ascending=False).index.values
cuisines_ratings = top30.groupby('cuisine')['rating'].mean().sort_values(ascending=False).values


# define fiuresource
source = ColumnDataSource(data=dict(
    x=list(cuisines_ratings),
    y=list(cuisines),
    colorMap = cividis30
))


# define figure
p = figure(y_range=cuisines[::-1], x_range=(3.5, cuisines_ratings.max()), 
           plot_width=450, plot_height=600, toolbar_location=None,
           title="Yelp ratings for the most popular NYC food categories",
           tools="hover", tooltips="@x{1.11}")

# plot bar chart
p.hbar(y='y', left=3.5, right='x', height=0.5, source=source, color='colorMap')

# figure parameters
p.xaxis.axis_label = "Avearage Yelp rating"
p.ygrid.grid_line_color = None
p.outline_line_color = None

# output
show(p)
save(p, 'cuisineYelpRatings.html')

/anaconda/envs/py36/lib/python3.6/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/anaconda/envs/py36/lib/python3.6/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'/Volumes/GoogleDrive/My Drive/PythonProjects/TooMuchFreeTime/Yelp API requests/cuisineYelpRatings.html'